In [1]:
import pandas as pd
import numpy as np


In [2]:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   43 bits physical, 48 bits virtual
CPU(s):                          32
On-line CPU(s) list:             0-31
Thread(s) per core:              2
Core(s) per socket:              16
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       AuthenticAMD
CPU family:                      23
Model:                           8
Model name:                      AMD Ryzen Threadripper 2950X 16-Core Processor
Stepping:                        2
Frequency boost:                 enabled
CPU MHz:                         2159.193
CPU max MHz:                     3500.0000
CPU min MHz:                     2200.0000
BogoMIPS:                        6986.19
Virtualization:                  AMD-V
L1d cache:                       512 KiB
L1i cache:                       1 MiB
L2 cache:             

In [3]:
!cat /proc/meminfo

MemTotal:       131763064 kB
MemFree:        118073040 kB
MemAvailable:   128197732 kB
Buffers:         1257924 kB
Cached:          9533344 kB
SwapCached:            0 kB
Active:          4783692 kB
Inactive:        7553000 kB
Active(anon):       3096 kB
Inactive(anon):  1560040 kB
Active(file):    4780596 kB
Inactive(file):  5992960 kB
Unevictable:          16 kB
Mlocked:              16 kB
SwapTotal:       2097148 kB
SwapFree:        2097148 kB
Dirty:              1196 kB
Writeback:             0 kB
AnonPages:       1545968 kB
Mapped:           661132 kB
Shmem:             28988 kB
KReclaimable:     603728 kB
Slab:             858396 kB
SReclaimable:     603728 kB
SUnreclaim:       254668 kB
KernelStack:       20048 kB
PageTables:        22444 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    67978680 kB
Committed_AS:    7398848 kB
VmallocTotal:   34359738367 kB
VmallocUsed:      119596 kB
VmallocChunk:          0 kB
Percpu:       

In [4]:
from scipy.fft import fft2, rfft2, hfft2

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../input/sample_submission.csv
../input/test.csv.gz
../input/train.csv.gz


In [5]:
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')
col = ['pixel%d'%i for i in range(784)]
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
X_train = train[col].values
X_test = test[col].values

In [8]:
X_train.shape

(42000, 784)

In [9]:
%%time
X_train_rffts = []

for i in range(X_train.shape[0]):
    x0 = X_train[i].reshape((28,28))
    x0_rfft = np.fft.rfft2(x0)
    X_train_rffts.append(x0_rfft.flatten())

CPU times: user 1.21 s, sys: 958 µs, total: 1.22 s
Wall time: 1.22 s


In [10]:
X_train_rffts = np.array(X_train_rffts)

In [11]:
X_train_rffts.shape

(42000, 420)

In [12]:
X_train_rffts = np.hstack([np.real(X_train_rffts), np.imag(X_train_rffts)])

In [13]:
X_train_rffts.shape

(42000, 840)

In [14]:
%%time
X_test_rffts = []

for i in range(X_test.shape[0]):
    x0 = X_test[i].reshape((28,28))
    x0_rfft = np.fft.rfft2(x0)
    X_test_rffts.append(x0_rfft.flatten())
    
X_test_rffts = np.array(X_test_rffts)
X_test_rffts = np.hstack([np.real(X_test_rffts), np.imag(X_test_rffts)])
X_test_rffts.shape

CPU times: user 895 ms, sys: 29.8 ms, total: 925 ms
Wall time: 924 ms


(28000, 840)

In [15]:
%%time
X_train_ffts = []

for i in range(X_train.shape[0]):
    x0 = X_train[i].reshape((28,28))
    x0_fft = fft2(x0)
    X_train_ffts.append(x0_fft.flatten())
    
X_train_ffts = np.array(X_train_ffts)
X_train_ffts.shape

CPU times: user 1.32 s, sys: 108 ms, total: 1.43 s
Wall time: 1.45 s


(42000, 784)

In [16]:
%%time
X_test_ffts = []

for i in range(X_test.shape[0]):
    x0 = X_test[i].reshape((28,28))
    x0_fft = fft2(x0)
    X_test_ffts.append(x0_fft.flatten())
    
X_test_ffts = np.array(X_test_ffts)
X_test_ffts = np.hstack([np.real(X_test_ffts), np.imag(X_test_ffts)])
X_test_ffts.shape

CPU times: user 941 ms, sys: 70.3 ms, total: 1.01 s
Wall time: 1.01 s


(28000, 1568)

In [17]:
%%time
X_train_hffts = []

for i in range(X_train.shape[0]):
    x0 = X_train[i].reshape((28,28))
    x0_hfft = hfft2(x0)
    X_train_hffts.append(x0_hfft.flatten())
    
X_train_hffts = np.array(X_train_hffts)
X_train_hffts.shape

CPU times: user 1.72 s, sys: 62.1 ms, total: 1.78 s
Wall time: 1.8 s


(42000, 1512)

In [18]:
%%time
X_test_hffts = []

for i in range(X_test.shape[0]):
    x0 = X_test[i].reshape((28,28))
    x0_hfft = hfft2(x0)
    X_test_hffts.append(x0_hfft.flatten())
    
X_test_hffts = np.array(X_test_hffts)
X_test_hffts.shape

CPU times: user 1.23 s, sys: 22.7 ms, total: 1.25 s
Wall time: 1.25 s


(28000, 1512)